- MySQL 커넥션 성능과 굉장히 밀접한 관련이 있는데 OS 커널파라미터중 네트워크와 밀접한 관련이 있음.

- MySQL이 보유할 수 있는 미해결 연결 요청 수
  - 이는 기본 MySQL 스레드가 매우 짧은 시간에 매우 많은 연결 요청을 받을 때 작동
  - 그런 다음 메인 스레드가 연결을 확인하고 새 스레드를 시작하는 데 약간의 시간이 걸림(아주 적은 시간이기는 하지만)
  - back_log 값은 MySQL이 일시적으로 새로운 요청에 대한 응답을 중지하기 전까지 이 짧은 시간 동안 얼마나 많은 요청이 쌓일 수 있는지 나타냄
- 짧은 시간 내에 많은 수의 연결이 예상되는 경우에만 이 값을 늘려야 함
  - 즉, 이 값은 들어오는 TCP/IP 연결에 대한 청취 대기열의 크기
  - 운영 체제에는 이 대기열의 크기에 대한 자체 제한이 있음
  - Unix Listen() 시스템 호출에 대한 매뉴얼 페이지에 더 자세한 내용이 존재
  - 이 변수의 최대값은 OS 설명서를 확인
  - back_log는 운영 체제 제한보다 높게 설정할 수 없음
  - 기본값은 허용된 백로그를 허용된 최대 연결 수에 맞게 조정할 수 있도록 하는 max_connections 값입니다

- MySQL main thread 는 신규 커넥션에 대해 검증하고 새로운 thread 를 할당하는 절차를 거침.
- 커넥션이 갑자기 많이 들어오면 병목 되어서 신규 커넥션 요청들이 대기해야 함.
- 무작정 모두 대기할 수는 없고 back_log 에서 정해 놓은 개수 만큼의 커넥션만 queue에 쌓아놓음.
  - 즉 TCP/IP 커넥션 요청에 대한 listen queue 사이즈.

> 신규 커넥션이 급격하게 증가할 경우, 신규 커넥션을 위해 Queue에 대기할 수 있는 커넥션의 개수  = 동시에 연결 요청 중인 클라이언트의 최대수


- 하지만 OS의 backlog 값이 있기 때문에 이 값을 넘어서면 안됨.

- 리눅스에서는 listen() system call 에 의해 결정.



- **listen 함수 예시**


```
#include <sys/socket.h>

int listen(int s, int backlog);

```




# **OS의 backlog**

**backlog는 2종류가 존재.**

- kernel backlog : 시스템 전체에 대한 backlog 를 제한. (tcp_max_syn_backlog)
- socket backlog : socket (port) 당 backlog를 제한. (somaxconn)



**커널 파라미터는 2종류가 존재.**

- /proc/sys/net/core/somaxconn = 128 (Socket Max Connection)

- /proc/sys/net/ipv4/tcp_max_syn_backlog = 8192



> 2가지 커널 파라미터 중에 MySQL은 어떤 것을 따라서 설정을 하면 될까?

> 결론은, MySQL back_log는 socket backlog 파라미터(somaxconn) 를 넘지 않도록 설정한다.


# **tcp_max_syn_backlog VS somaxconn**

**파라미터의 미세한 차이**


> 둘다 backlog 제한하는 것처럼 나오는데.

- tcp_max_syn_backlog 는 시스템 전체의 backlog 설정
- somaxconn 는 포트당 할당가능한 backlog



**kernel backlog (tcp_max_sync_backlog)**
- 시스템 전체 backlog 제한하는 값, TCP 3way 통신할 때 아직 ACK받지 못한 커넥션들이 해당.
  - Client -> Kernel : SYN OK
  - Kernel -> Client : SYN / ACK OK
  - Client -> Kernel : ACK ..... (ACK를 아직 못받음)

**socket backlog (somaxconn)**
- listen() 시스템콜을 사용할 떄 backlog를 제한하는 값



**각 커널 파라미터별 장애 영향도**

- somaxconn 값이 작으면 apache 구동 자체에 문제 발생
- tcp_max_syn_backlog 값이 작으면 Sending cookies. 메시지 발생
- 너무 작으면 클라이언트 Benchmarking mapoo.net (be patient)…apr_socket_recv: Connection reset by peer (104) 메시지 출력



**TCP 연결**

- Server Socket
  - socket() -> bind() -> listen() -> accept()
1. 소켓 생성
2. 생성된 소켓을 로컬 주소에 바인딩
3. 들어오는 연결을 accept (tcp 3-way handshake 이후에 발생)


**parameter info**

|설명| 값|
|--|--|
|커맨드 라인 명령어 정보|--back-log=#|
|시스템 변수|back_log|
|범위|Global|
|동적여부|N|
|SET_VAR. 힌트 적용 여부 : No|No|
|형식|Integer|
|기본값|-1 (자동크가. 리터럴값에 할당하면 안됩니다.)|
|최소값|1|
|최대값|65535|
